# Generate Dummy Event Submissions
This notebook generates 1–10 dummy `event_submissions` per non-GPA subcategory for all students.

In [ ]:
from __future__ import annotations
import os, random, datetime as dt
from typing import Any, Dict, List
from dotenv import load_dotenv
load_dotenv()
from supabase import Client

# Configure Supabase client
SUPABASE_URL = os.getenv('SUPABASE_URL') or os.getenv('NEXT_PUBLIC_SUPABASE_URL')
SUPABASE_KEY = os.getenv('SUPABASE_SERVICE_ROLE_KEY') or os.getenv('SUPABASE_ANON_KEY')
if not SUPABASE_URL or not SUPABASE_KEY:
    raise RuntimeError('Missing Supabase credentials; set SUPABASE_URL and SUPABASE_SERVICE_ROLE_KEY')
sb = Client(SUPABASE_URL, SUPABASE_KEY)

# GPA subcategories to skip normalization generation
GPA_SUBCATEGORY_IDS = {
    'f50830fe-b820-4223-89e2-e69241b459af',
    '8d13f1b9-33e1-4a62-be45-488a6834112f',
    'd1d972a4-2484-4b9a-a53c-0b63bb2e952c',
}

def fetch_students() -> List[Dict[str, Any]]:
    return sb.table('students').select('id').execute().data or []

def fetch_subcategories() -> List[Dict[str, Any]]:
    return sb.table('subcategories').select('id,name').execute().data or []

def fetch_events_by_subcategory() -> Dict[str, List[str]]:
    rows = sb.table('event_instances').select('id, subcategory_id, is_active').eq('is_active', True).execute().data or []
    by_sub: Dict[str, List[str]] = {}
    for r in rows:
        sid = r.get('subcategory_id')
        if not sid:
            continue
        by_sub.setdefault(sid, []).append(r['id'])
    return by_sub

# Payload generators aligned with packages/types/src/event-submissions.ts
def gen_attendance():
    return { 'submission_type': 'attendance', 'status': random.choice(['present','absent','excused']), 'notes': 'auto-attendance' }

def gen_monthly_check(sub: str):
    return { 'submission_type': sub, 'status': random.choice(['involved','not_involved']), 'notes': f'auto-{sub}' }

def gen_community_service():
    return { 'submission_type': 'community_service', 'hours': random.randint(1,6), 'organization': 'Community Org', 'supervisor_name': 'Jane Doe', 'supervisor_contact': 'jane@example.com', 'description': 'Volunteer task', 'date_of_service': dt.date.today().isoformat(), 'notes': 'auto-service' }

def gen_credentials():
    return { 'submission_type': 'credentials', 'credential_name': 'Cert X', 'granting_organization': 'Org Y', 'description': 'Awarded', 'date_of_credential': dt.date.today().isoformat(), 'assigned_points': random.randint(5,30), 'notes': 'auto-credential' }

def gen_job_promotion():
    return { 'submission_type': 'job_promotion', 'promotion_title': 'Assistant → Senior', 'organization': 'ACU Blueprint', 'supervisor_name': 'John Doe', 'supervisor_contact': 'john@example.com', 'description': 'Promoted', 'date_of_promotion': dt.date.today().isoformat(), 'assigned_points': random.randint(10,50), 'notes': 'auto-promotion' }

def gen_participation_points():
    return { 'submission_type': 'participation', 'points': random.randint(1,5), 'notes': 'auto-points' }

def gen_lions_games():
    return { 'submission_type': 'lions_games', 'assigned_points': random.randint(1,10), 'notes': 'auto-lions' }

# Map subcategory names to generators
GEN_MAP = {
  'fellow_friday_attendance': gen_attendance,
  'gbe_attendance': gen_attendance,
  'chapel_attendance': gen_attendance,
  'small_group_involvement': lambda: gen_monthly_check('small_group'),
  'dream_team_involvement': lambda: gen_monthly_check('dream_team'),
  'community_service_hours': gen_community_service,
  'credentials_certifications': gen_credentials,
  'job_promotion_opportunities': gen_job_promotion,
  'chapel_participation': gen_participation_points,
  'company_team_building': gen_participation_points,
  'fellow_friday_participation': gen_participation_points,
  'gbe_participation': gen_participation_points,
  'lions_games_involvement': gen_lions_games,
}

def generate_for_student(student_id: str, sub_by_events: Dict[str, List[str]], sub_list: List[Dict[str, Any]]):
    # Seed used event_ids for this student to enforce uniqueness (student_id, event_id)
    used_resp = sb.table('event_submissions').select('event_id').eq('student_id', student_id).execute()
    used_event_ids = {r['event_id'] for r in (used_resp.data or []) if r.get('event_id')}

    for sub in sub_list:
        sid, name = sub['id'], sub['name']
        if sid in GPA_SUBCATEGORY_IDS:
            continue
        gen = GEN_MAP.get(name)
        if not gen:
            continue
        base_events = list(sub_by_events.get(sid) or [])
        # Remove already-used events for this student
        available = [e for e in base_events if e not in used_event_ids]
        if not available:
            continue
        random.shuffle(available)
        # Choose up to remaining capacity
        max_new = min(len(available), random.randint(1, 10))
        take = available[:max_new]
        for event_id in take:
            payload = gen()
            sb.table('event_submissions').insert({
                'event_id': event_id,
                'student_id': student_id,
                'submitted_by': student_id,
                'submission_data': payload,
                'submitted_at': dt.datetime.utcnow().isoformat(),
                'subcategory_id': sid,
                'needs_approval': False,
                'approval_status': 'approved'
            }).execute()
            used_event_ids.add(event_id)

def run():
    students = fetch_students()
    sub_list = fetch_subcategories()
    events_by_sub = fetch_events_by_subcategory()
    for s in students:
        generate_for_student(s['id'], events_by_sub, sub_list)
    return {'students': len(students), 'subcategories': len(sub_list)}

run()


/var/folders/d7/6z97jd4151v1gx2srcw4n0740000gp/T/ipykernel_98097/947617312.py:108: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'submitted_at': dt.datetime.utcnow().isoformat(),


{'students': 22, 'subcategories': 17}